# Библиотеки

In [1]:
import pandas as pd
from tqdm.auto import tqdm

import re
import requests
from urllib.parse import urlencode
from  pandasql import sqldf

In [2]:
import folium
from folium.plugins import MarkerCluster

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
import math
from folium import plugins

# Добавление геолокации

In [ ]:
df_final_HH = pd.read_csv('Вакансии//df_final_HH.csv',index_col=0)

In [ ]:
df_final_Realty = pd.read_csv('Квартиры//df_final_Realty.csv',)

In [ ]:
# функуия которая всегда находит location
def extract_lat_lng(address_or_postalcode, data_type = 'json'):
    endpoint = f"https://maps.googleapis.com/maps/api/geocode/{data_type}"
    params = {"address": address_or_postalcode, "key": api_key}
    url_params = urlencode(params)
    url = f"{endpoint}?{url_params}"
    r = requests.get(url)
    if r.status_code not in range(200, 299): 
        return {}
    latlng = {}
    try:
        latlng = r.json()['results'][0]['geometry']['location']
    except:
        pass
    return latlng.get("lat"), latlng.get("lng")

In [ ]:
# чтобы не грузить в гугл дубликаты будем их удалять 
# итоговый датафрейм полуичится как результат обЪединения 2-x
def df_of_location(df):
    box = []
    for i in tqdm(df['tech_address'].unique()):
        loc = extract_lat_lng(i)
        box.append([i, loc[0], loc[1]])
    df_with_loc =  pd.DataFrame(box,columns =['tech_address', 'lat', 'lon'])
    return df.merge(df_with_loc, left_on='tech_address',right_on='tech_address', how = 'left')

In [ ]:
# прикручиваем долготу и ширину
df_final_Realty = df_of_location(df_final_Realty)

In [ ]:
df_final_HH = df_of_location(df_final_HH)

In [ ]:
df_final_HH.dropna(inplace = True)
df_final_Realty.dropna(inplace = True)

In [ ]:
df_final_Realty.to_csv('df_final_Realty_with_loc.csv')
df_final_HH.to_csv('df_final_HH_with_loc.csv')

# Загрузка

In [5]:
df_final_HH_with_loc = pd.read_csv('df_final_HH_with_loc.csv',index_col=0)
df_final_Realty_with_loc = pd.read_csv('df_final_Realty_with_loc.csv',index_col=0)

In [6]:
df_final_HH_with_loc = df_final_HH_with_loc.query('experience == "От 1 года до 3 лет"').reset_index(drop = True)

In [7]:
# на карту попадаем много дубликатов
# так как на одну и ту же вакансию могут быть несколько объявлений
df_final_HH_with_loc.drop_duplicates(subset = ['alternate_url'], inplace = True)

In [8]:
df_final_HH_with_loc = sqldf("select * from df_final_HH_with_loc where \
            1==1 \
            and name not like '%Системный%'  \
            and name not like '%cистемный%' \
            and name not like '%1C%'\
            and name not like '%Врач%' \
                             ")

In [9]:
df_final_Realty_with_loc.rename(columns = {'type of real estate': 'type_of_real_estate'}, inplace = True)

In [10]:
# переопределим датафрейм с  вакансиями

In [11]:
df_final_HH_with_loc2= df_final_HH_with_loc.query('Название_региона in ("Москва", "Санкт-Петербург", "Приморский край")').reset_index(drop = True)

In [12]:
dic ={'Москва':'moskva'
     ,"Санкт-Петербург":'sankt-peterburg'
      , "Приморский край":'vladivostok'
     }

In [13]:
df_final_HH_with_loc2['city'] = df_final_HH_with_loc2['Название_региона'].map(dic)

In [14]:
# функция поиска расстояни по прямой
def distance_by_line(lat1, lat2 ,lon1, lon2 ):
    R=6371 #Earth's radius
    sin1=math.sin((lat1-lat2)/2)
    sin2=math.sin((lon1-lon2)/2)
    return 2*R*math.asin(math.sqrt(sin1*sin1+sin2*sin2*math.cos(lat1)*math.cos(lat2)))

In [15]:
def plotting_map(df_with_loc, df_with_loc2,  signature, name, kind = 'all'):
    
    '''
    kind = [all, квартра, дом]
    '''
    if kind == ' квартира':
        condition = "where type_of_real_estate like '%комн%'"
    elif kind == 'дом':
        condition = "where type_of_real_estate not like '%комн%'"
    
    else:
        condition = ""
        
    df_with_loc2 =  sqldf("select * from df_with_loc2 " + condition)
    
    
    schoolheatmap = folium.Map(location=df_with_loc[["lat", "lon"]].mean().to_list(), zoom_start=2)
  
    latitude    = df_with_loc2['lat']
    longitude   = df_with_loc2['lon']

    folium.plugins.HeatMap(list(zip(latitude,
                                    longitude)),
                           radius=30,
                           blur=20).add_to(schoolheatmap)
    plugins.Fullscreen(position='topright').add_to(schoolheatmap)
    
    for i,r in df_with_loc.iterrows():
        location = (r["lat"], r["lon"])
        folium.Marker(location=location 
                      ,popup = r[name] + ' ' + r['alternate_url'] # подпись вакансии 
                      ,icon= folium.Icon(color="red", icon="info-sign") if r["name_family"] != "Иван" else folium.Icon(icon="blue")
                      ,tooltip = f'{r[signature]/10**6:.2f}'+ ' млн.' if signature == "price" else \
                                f'{r[signature]/10**3:.2f}'+ ' тыс.'
                     )\
                      .add_to(schoolheatmap)
    return schoolheatmap

In [16]:
plotting_map(df_final_HH_with_loc,
            df_final_Realty_with_loc, 
            name = 'name',
            signature = 'salary.from',
            kind = 'all')

In [16]:
def find_place(kind = 'all'):
    '''
    kind = [all, квартра, дом]
    '''
    if kind == ' квартира':
        condition = "where type_of_real_estate like '%комн%'"
    elif kind == 'дом':
        condition = "where type_of_real_estate not like '%комн%'"
    
    else:
        condition = ""
        
    df_final_Realty_with_loc =  sqldf("select * from df_final_Realty_with_loc " + condition) 
    box = []
    for city in df_final_Realty_with_loc.city.unique():
        for person in df_final_HH_with_loc2.name_family.unique():
            for i,r_realty in df_final_Realty_with_loc.query('city == @city').iterrows():
                for j ,r_hh in df_final_HH_with_loc2.query('city == @city and name_family == @person').iterrows():
                    distance = distance_by_line(r_realty['lat'], r_hh['lat'] ,r_realty['lon'], r_hh['lon'] )
                    box.append([city, r_realty['tech_address'], r_hh['tech_address'], person, distance])


    dictance_table = pd.DataFrame(box, columns =['city', 'i_house', 'j_vacan', 'person', 'distance'])

    tabl_tem = dictance_table.groupby(['person', 'city','i_house']).agg({'distance' : 'min'}).reset_index()

    return tabl_tem.pivot_table(index=['city', 'i_house']  ,values='distance',aggfunc='sum') \
        .fillna(0) \
        .reset_index() \
        .sort_values(by =['city', 'distance']) \
        .groupby('city') \
        . head(3) \
        .reset_index(drop = True)

In [17]:
find_place(kind = 'квартира')

,city,i_house,distance
0,moskva,"moskva \nновостройка, ЖК «Метрополия» 2024 г.,...",58.367574
1,moskva,"moskva \nновостройка, ЖК «Метрополия» 2024 г.,...",58.367574
2,moskva,"moskva \nновостройка, ЖК «Метрополия» 2024 г.,...",58.367574
3,sankt-peterburg,"sankt-peterburg улица Лагоды, 5",84.841812
4,sankt-peterburg,"sankt-peterburg проспект Медиков, 10к6",112.440999
5,sankt-peterburg,"sankt-peterburg проспект Энергетиков, 28к7",112.667058
6,vladivostok,"vladivostok 1-я Морская улица, 6",7.396889
7,vladivostok,"vladivostok Светланская улица, 157",18.666082
8,vladivostok,"vladivostok улица Полонского, 29",22.786280
